# Import libs, read file, create test_lst

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import math

In [6]:
df = pd.read_csv("sthlm/id_sold_sthlm.csv")

In [8]:
lst_gator = df.values.tolist()

# Skapa test_lst med id

- I wan´t to avoid looping 58000 times every time I'm trying out something new
- Therefore I'm creating a testing environment to try out new things more efficiently

**No. of loops for testing environment?**

<u>High number</u>
- Pro: Higher probability of finding different special cases
- Con: More time

<u>Low number</u>
- Pro: Less time
- Con: Lower probability of finding different special cases

**test_lst_gator**
- test_lst_gator is the name of the list I will loop over when trying out new functionality
- It should give ~500 loops for min 5 different streets
- Any conditional logic  which yields such a list is ok

In [24]:
mask_min = df.antal_sålda >= 58
mask_max = df.antal_sålda <= 60

In [25]:
test_lst_gator = df.loc[mask_min & mask_max].values.tolist()

In [26]:
len(test_lst_gator)

10

In [27]:
a=0
for i in test_lst_gator:
    a += i[0]
print(a)

594


# Test-loop for test_lst

- In this loop I'm trying out 1 functionality at a time
- 1 functionality is 1 scraping 1 piece of information from a sold apartment

In [28]:
temp_lst = []

for i in test_lst_gator:    
    page_num = math.ceil(int(i[0])/50)
    for j in range(1, page_num+1):
        url = f"https://www.hemnet.se/salda/bostader?item_types%5B%5D=bostadsratt&location_ids%5B%5D={i[2]}&page={j}&sold_age=all"
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, ("lxml"))

        container = soup.select("li.sold-results__normal-hit")

        for x in container:
            gata = i[1]
            gata_re = re.findall(r"[-] +[A-Za-zåäöÅÄÖ\s]+[,]", gata)
            gata_re_strip = gata_re[0].strip(",-").strip()
            temp_lst.append(gata_re_strip)
print(len(temp_lst))

593


# Main-loop for test_lst

- I this loop I'm running all functionality for the objects in test_lst
- All functionality is scraping all pieces of information from each object

In [119]:
adress_lst = []
omrade_lst = []
kvm_lst = []
rum_lst = []
maklare_lst = []
avgift_lst = []
slutpris_lst = []
datum_lst = []
change_lst = []
gata_id_lst = []
gata_lst = []
stockholm_lst = []


for i in test_lst:    
    page_num = math.ceil(int(i[0])/50)
    for j in range(1, page_num+1):
        url = f"https://www.hemnet.se/salda/bostader?item_types%5B%5D=bostadsratt&location_ids%5B%5D={i[2]}&page={j}&sold_age=all"
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, ("lxml"))

        container = soup.select("li.sold-results__normal-hit")

        for x in container: 
            
            # Lägg till kolumner från listan 
            
            ## id
            gata_id_lst.append(i[2])
            
            ## gata
            gata = i[1]
            gata_re = re.findall(r"[-] +[A-Za-zåäöÅÄÖ\s]+[,]", gata)
            gata_re_strip = gata_re[0].strip(",-").strip()
            gata_lst.append(gata_re_strip)
            
            ## Stockholms kommun
            if "Stockholms kommun" in i[1]:
                stockholm_lst.append("Stockholms kommun")
            else:
                stockholm_lst.append(np.nan)
            
            # Lägg till kolumner från Hemnet

            ## adress
            adress = [x.text for x in x.select("span.item-result-meta-attribute-is-bold")]
            if adress:
                adress_lst.append(adress[0])
            else:
                adress.lst.append(np.nan)
            
            ## område
            omr = [x.text for x in x.select("span.item-link")]
            if len(omr) == 1:
                omrade_lst.append(np.nan)    
            else:
                omrade_lst.append(omr[1].strip())
                
            ## KVM
            kvm = [x.text for x in x.select("div.sold-property-listing__subheading")]    
            kvm = re.findall("[0-9]+", kvm[0])

            if len(kvm) <= 1:
                kvm_lst.append(np.nan) 
            elif len(kvm) == 2:
                kvm = kvm[0]
                kvm_lst.append(kvm)
            elif len(kvm) ==3:
                kvm = ".".join(kvm[0:2])
                kvm_lst.append(kvm)
            elif len(kvm) ==4:
                kvm = ".".join(kvm[0:2])
                kvm_lst.append(kvm)
            else:
                kvm_lst.append(np.nan)     
            
            ## Rum
            rum = [x.text for x in x.select("div.sold-property-listing__subheading")]   
            rum = re.findall("[0-9]+", rum[0])

            if not rum:
                rum_lst.append(np.nan)
            elif len(rum) == 1 and int(rum[0]) < 10:
                rum_lst.append(rum[0])
            elif len(rum) == 1 and int(rum[0]) > 10:
                rum_lst.append(np.nan)
            elif len(rum) == 2:
                rum = rum[1]
                rum_lst.append(rum)
            elif len(rum) == 3:
                rum = ".".join(rum[2])
                rum_lst.append(rum)
            elif len(rum) ==4:
                rum = ".".join(rum[2:4])
                rum_lst.append(rum)
            else:
                rum_lst.append(np.nan)
                
            ## Mäklare
            maklare = [x.text.strip() for x in x.select("div.sold-property-listing__broker")]
            if maklare:
                maklare_lst.append(maklare[0])
            else:
                maklare_lst.append(np.nan)
                
            ## Avgift
            avgift = [x.text.strip() for x in x.select("div.sold-property-listing__fee")]
            if len(avgift) == 1:
                avgift = re.findall("[0-9]+", avgift[0])
                avgift_lst.append("".join(avgift))
            else: 
                avgift_lst.append(np.nan)
                
            ## Slutpris
            slutpris = [re.findall("[0-9]+", x.text) for x in x.select("span.sold-property-listing__subheading")]
            if slutpris:
                slutpris_lst.append("".join(slutpris[0]))
            else:
                slutpris_lst.append(np.nan)
                
            ## Datum
            datum = [x.text.strip() for x in x.select("div.sold-property-listing__sold-date")]
            if datum:
                datum_lst.append(datum[0])
            else:
                datum_lst.append(np.nan)
                
            ## Måste ligga längs ned så som koden ser ut nu. Annars förstör continue
            ## Prisförändring i % 
            change = [re.findall("[0-9+-]+", x.text) for x in x.select("div.sold-property-listing__price-change")]
            if not change:
                change_lst.append(0)
                continue
            if not change[0]:
                change_lst.append(0)
                continue
            else:
                change_lst.append(change[0][0])

In [ ]:
print(len(adress_lst))
print(len(omrade_lst))
print(len(kvm_lst))
print(len(rum_lst))
print(len(maklare_lst))
print(len(avgift_lst))
print(len(slutpris_lst))
print(len(datum_lst))
print(len(change_lst))
print(len(gata_id_lst))
print(len(gata_lst))
print(len(stockholm_lst))

In [121]:
# Zippa listor 
comp_lst = list(zip(adress_lst, omrade_lst, kvm_lst, rum_lst, maklare_lst,
                    avgift_lst, slutpris_lst, datum_lst, change_lst, gata_id_lst, gata_lst, stockholm_lst))

In [122]:
# Skapa df

df_sold = pd.DataFrame(data = comp_lst, columns = ["adress", "omrade", "kvm", "rum",
                                                  "maklare", "avgift", "slutpris", "datum",
                                                    "prisförändring", "gata_id_lst", "gata_lst", "stockholm_lst"])

In [31]:
df_sold

# Main-loop för main_lst

In [29]:
len(lst_gator)

1273

In [ ]:
adress_lst = []
omrade_lst = []
kvm_lst = []
rum_lst = []
maklare_lst = []
avgift_lst = []
slutpris_lst = []
datum_lst = []
change_lst = []
gata_id_lst = []
gata_lst = []
stockholm_lst = []


for i in lst_gator:    
    page_num = math.ceil(int(i[0])/50)
    for j in range(1, page_num+1):
        url = f"https://www.hemnet.se/salda/bostader?item_types%5B%5D=bostadsratt&location_ids%5B%5D={i[2]}&page={j}&sold_age=all"
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, ("lxml"))

        container = soup.select("li.sold-results__normal-hit")

        for x in container: 
            
            # Lägg till kolumner från listan 
            
            ## id
            gata_id_lst.append(i[2])
            
            ## gata
            gata = i[1]
            gata_re = re.findall(r"[-] +[A-Za-zåäöÅÄÖ\s]+[,]", gata)
            if gata_re:                
                gata_re_strip = gata_re[0].strip(",-").strip()
                gata_lst.append(gata_re_strip)
            else: 
                gata_lst.append(np.nan)
            
            ## Stockholms kommun
            if "Stockholms kommun" in i[1]:
                stockholm_lst.append("Stockholms kommun")
            else:
                stockholm_lst.append(np.nan)
            
            # Lägg till kolumner från Hemnet

            ## adress
            adress = [x.text for x in x.select("span.item-result-meta-attribute-is-bold")]
            if adress:
                adress_lst.append(adress[0])
            else:
                adress.lst.append(np.nan)
            
            ## område
            omr = [x.text for x in x.select("span.item-link")]
            if len(omr) == 1:
                omrade_lst.append(np.nan)    
            else:
                omrade_lst.append(omr[1].strip())
                
            ## KVM
            kvm = [x.text for x in x.select("div.sold-property-listing__subheading")]    
            kvm = re.findall("[0-9]+", kvm[0])

            if len(kvm) <= 1:
                kvm_lst.append(np.nan) 
            elif len(kvm) == 2:
                kvm = kvm[0]
                kvm_lst.append(kvm)
            elif len(kvm) ==3:
                kvm = ".".join(kvm[0:2])
                kvm_lst.append(kvm)
            elif len(kvm) ==4:
                kvm = ".".join(kvm[0:2])
                kvm_lst.append(kvm)
            else:
                kvm_lst.append(np.nan)     
            
            ## Rum
            rum = [x.text for x in x.select("div.sold-property-listing__subheading")]   
            rum = re.findall("[0-9]+", rum[0])

            if not rum:
                rum_lst.append(np.nan)
            elif len(rum) == 1 and int(rum[0]) < 10:
                rum_lst.append(rum[0])
            elif len(rum) == 1 and int(rum[0]) > 10:
                rum_lst.append(np.nan)
            elif len(rum) == 2:
                rum = rum[1]
                rum_lst.append(rum)
            elif len(rum) == 3:
                rum = ".".join(rum[2])
                rum_lst.append(rum)
            elif len(rum) ==4:
                rum = ".".join(rum[2:4])
                rum_lst.append(rum)
            else:
                rum_lst.append(np.nan)
                
            ## Mäklare
            maklare = [x.text.strip() for x in x.select("div.sold-property-listing__broker")]
            if maklare:
                maklare_lst.append(maklare[0])
            else:
                maklare_lst.append(np.nan)
                
            ## Avgift
            avgift = [x.text.strip() for x in x.select("div.sold-property-listing__fee")]
            if len(avgift) == 1:
                avgift = re.findall("[0-9]+", avgift[0])
                avgift_lst.append("".join(avgift))
            else: 
                avgift_lst.append(np.nan)
                
            ## Slutpris
            slutpris = [re.findall("[0-9]+", x.text) for x in x.select("span.sold-property-listing__subheading")]
            if slutpris:
                slutpris_lst.append("".join(slutpris[0]))
            else:
                slutpris_lst.append(np.nan)
                
            ## Datum
            datum = [x.text.strip() for x in x.select("div.sold-property-listing__sold-date")]
            if datum:
                datum_lst.append(datum[0])
            else:
                datum_lst.append(np.nan)
                
            ## Måste ligga längs ned så som koden ser ut nu. Annars förstör continue
            ## Prisförändring i % 
            change = [re.findall("[0-9+-]+", x.text) for x in x.select("div.sold-property-listing__price-change")]
            if not change:
                change_lst.append(0)
                continue
            if not change[0]:
                change_lst.append(0)
                continue
            else:
                change_lst.append(change[0][0])
                
print(len(adress_lst))
print(len(omrade_lst))
print(len(kvm_lst))
print(len(rum_lst))
print(len(maklare_lst))
print(len(avgift_lst))
print(len(slutpris_lst))
print(len(datum_lst))
print(len(change_lst))
print(len(gata_id_lst))
print(len(gata_lst))
print(len(stockholm_lst))

# Inspecting df

In [129]:
# Zippa listor 
comp_lst = list(zip(adress_lst, omrade_lst, kvm_lst, rum_lst, maklare_lst,
                    avgift_lst, slutpris_lst, datum_lst, change_lst, gata_id_lst, gata_lst, stockholm_lst))

In [130]:
# Skapa df

df_sold = pd.DataFrame(data = comp_lst, columns = ["adress", "omrade", "kvm", "rum",
                                                  "maklare", "avgift", "slutpris", "datum",
                                                    "prisförändring", "gata_id_lst", "gata_lst", "stockholm_lst"])